## INSTALL DEPENDENCIES

In [ ]:
%pip install presidio_analyzer
%pip install presidio_anonymizer
%pip install transformers
%pip install pandas
%pip install spacy
%pip install torch
%pip install seqeval
%pip install spacy-transformers

### INSTALL SIMPLE SPACY MODEL

In [ ]:
!python -m spacy download en_core_web_sm

## INSTALL COMPLEX SPACY MODEL (ONLY IF YOU USE THIS INSTEAD OF BERT)

In [ ]:
!python -m spacy download en_core_web_lg

## IMPORTS

In [1]:
from presidio_analyzer import AnalyzerEngine, RecognizerResult, RecognizerRegistry
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
import pandas as pd
from transformers_rec import (
    TransformersRecognizer,
    BERT_DEID_CONFIGURATION,
)
import logging
from presidio_anonymizer.entities import OperatorConfig
from typing import List
from spacy import displacy
import csv
import json
from tqdm import tqdm
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
from seqeval.metrics import recall_score
import re
import string
import warnings
import spacy

/Users/gonzalo.zelinka/Desktop/PII_POC/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## CREATE ANALYZER AND ANONYMIZE FUNCTION

In [2]:
def analyzer_engine(model_path):
  """Return AnalyzerEngine.
    :param model_path: Which model to use for NER:
        "obi/deid_roberta_i2b2",
        "en_core_web_lg"
    """
  registry = RecognizerRegistry()
  registry.load_predefined_recognizers()
  if model_path == "en_core_web_lg":
    nlp_configuration = {
        "nlp_engine_name": "spacy",
        "models": [{"lang_code": "en", "model_name": "en_core_web_lg"}],
    }
  elif model_path == "custom_P_E":
    # nlp = spacy.load("models/P&E-model3/model-best")
    # registry.add_recognizer(SpacyRecognizer(nlp))
    nlp_configuration = {
        "nlp_engine_name": "spacy",
        "models": [{"lang_code": "en", "model_name": "models/P&E-model3/model-best"}],
    }
  else:
    # Using a small spaCy model + a HF NER model
    transformers_recognizer = TransformersRecognizer(model_path=model_path)
    if model_path == "obi/deid_roberta_i2b2":
      transformers_recognizer.load_transformer(**BERT_DEID_CONFIGURATION)
    else:
      transformers_recognizer.load_transformer(**BERT_PII_CONFIGURATION)
    # Use small spaCy model, no need for both spacy and HF models
    # The transformers model is used here as a recognizer, not as an NlpEngine
    nlp_configuration = {
      "nlp_engine_name": "spacy",
      "models": [{"lang_code": "en", "model_name": "en_core_web_sm"}],
    }
    registry.add_recognizer(transformers_recognizer)

  nlp_engine = NlpEngineProvider(nlp_configuration=nlp_configuration).create_engine()

  analyzer = AnalyzerEngine(nlp_engine=nlp_engine, registry=registry)
  return analyzer


In [3]:
def analyze(analyzer, **kwargs):
    """Analyze input using Analyzer engine and input arguments (kwargs)."""
    if "entities" not in kwargs or "All" in kwargs["entities"]:
        kwargs["entities"] = None
    return analyzer.analyze(**kwargs)

In [4]:
def anonymize(text: str, analyze_results: List[RecognizerResult]):
    """Anonymize identified input using Presidio Anonymizer.
    :param text: Full text
    :param analyze_results: list of results from presidio analyzer engine
    """
    operator_config = {"lambda": lambda x: x}
    operator = "custom"
    res = AnonymizerEngine().anonymize(
        text,
        analyze_results,
        operators={"DEFAULT": OperatorConfig(operator, operator_config)},
    )
    return res

## INITIAL CONFIG FOR THE ANALYZER

In [5]:
analyzer = analyzer_engine("obi/deid_roberta_i2b2") # "en_core_web_lg" or "obi/deid_roberta_i2b2" or "custom_P_E"

In [43]:
threshold = 0.40
entities = ["PERSON", "LOCATION", "PHONE_NUMBER", "EMAIL_ADDRESS","CREDIT_CARD", "US_SSN"]

In [6]:
def create_obj(an_r, text):
    """Show results of analyze() in a dataframe."""
    ents = []
    for r in an_r:
      info = r.to_dict()
      ent ={ "start": info["start"], 
              "end": info['end'], 
              "confidence": info['score'], 
              "entity": info['entity_type'], 
              "text": text[info["start"]:info["end"]]} 
      ents.append(ent)
    return ents


def model_results(csv_path, json_path, entities, threshold, analyzer,columns, check_overlaps=False):
  results = []
  df = pd.read_csv(csv_path, encoding="ISO-8859-1",header=0, names=columns)
  # file = open(csv_path, 'r', encoding="ISO-8859-1")
  # reader = csv.reader(file)
  # rows = list(reader)
  # for row in tqdm(rows, total=len(rows)):
  for index, row in tqdm(df.iterrows(), total=len(df)):
    # id = row.PVID
    # text = row.CONTENT
    text = row[0]
    analyze_results = analyze(
      analyzer=analyzer,
      text=text,
      entities= entities,
      language="en",
      score_threshold=threshold,
    )
    if check_overlaps: # return only entities without overlaps (resolved from presidio) and prediction.
      text_anon = anonymize(text, analyze_results)
      text_anon = sorted(text_anon.items, key=lambda x: x.start)
      result = []
      for i, res in enumerate(text_anon):
          result.append({"start": res.start, "end": res.end, "entity": res.entity_type, "text": res.text})
          
    else: # return all entities with overlaps and prediction. 
      result = create_obj(analyze_results, text)
    # results.append({"PVID": id, "TEXT": text, "ENTITIES": result})
    results.append({"TEXT": text, "ENTITIES": result})
  fp=open(json_path,'w', encoding="ISO-8859-1") # output file
  json.dump(results, fp)  

## TEST SIMPLE DATA

In [47]:
def annotate(text: str, analyze_results: List[RecognizerResult]):
    """
    Highlights every identified entity on top of the text.
    :param text: full text
    :param analyze_results: list of analyzer results.
    """
    ents = []

    # Use the anonymizer to resolve overlaps
    results = anonymize(text, analyze_results)
    # sort by start index
    results = sorted(results.items, key=lambda x: x.start)
    for i, res in enumerate(results):
        ents.append({"start": res.start, "end": res.end, "label": res.entity_type, "text": res.text})
    return [{"text": text, "ents": ents}]

In [46]:
def show_results(an_r, text, return_analyzer_results=False):
    """Show results of analyze() in a dataframe."""
    df = pd.DataFrame.from_records([r.to_dict() for r in an_r])
    df["text"] = [text[res.start: res.end] for res in an_r]
    df_subset = df[["entity_type", "text", "start", "end", "score"]].rename(
        {
            "entity_type": "Entity type",
            "text": "Text",
            "start": "Start",
            "end": "End",
            "score": "Confidence",
        },
        axis=1,
    )
    df_subset["Text"] = [text[res.start: res.end] for res in an_r]
    #  In analysis_explanation_df there are more columns than in df_subset with more information. 
    if return_analyzer_results:
      analysis_explanation_df = pd.DataFrame.from_records(
          [r.analysis_explanation.to_dict() for r in an_r]
      )
    # df_subset = pd.concat([df_subset, analysis_explanation_df], axis=1)
    result = annotate(text, an_r)
    return df_subset.reset_index(drop=True), result
  

In [41]:
text="My name is Gonzalo Zelinka, I'm from Argentina"

In [44]:
analyze_results = analyze(
    analyzer=analyzer,
    text=text,
    entities= entities,
    language="en",
    score_threshold=threshold,
)

In [48]:
frame, sentence = show_results(analyze_results, text)
# print(sentence)
displacy.render(sentence, style="ent", manual=True)
display(frame)

,Entity type,Text,Start,End,Confidence
0,PERSON,Gonzalo,11,18,1.00
1,PERSON,Zelinka,19,26,1.00
2,LOCATION,Argentina,37,46,1.00
3,PERSON,Gonzalo Zelinka,11,26,0.85


## GENERATE RESULTS FROM COMPLEX DATA

#### CHANGE DIRECTORY

In [ ]:
import os

os.getcwd()
#Change this
os.chdir("")
os.getcwd()

### RUN MODEL

In [ ]:
model_results("testing-data/product_reviews7_test_sentences.csv", "testing-data/product_reviews8.json", entities, threshold, analyzer, True)

## EVALUATE MODEL

### EXTRACT THE DATA TRUTH

In [2]:
def get_span_indx(
    labels: List[str],
    words: List[str],
    sentence: str
) -> List[tuple]:
    """Gets span starts and ends for Spacy spancat component.
        
        Returns list of tuples where the first element of the 
        tuple is the span start, the second element of the tuple
        is the span end and the third element of the tuple is
        the span category. 
    """
    #gets list of indices corresponding to labelled words 
    label_indx = []
    temp_list = []

    for i, l in enumerate(labels):
        if l != 'O':
            temp_list.append(i)
        else:
            label_indx.append(temp_list)
            temp_list = []    
        if i == len(labels) - 1:
            label_indx.append(temp_list)

    clean_label_indx = [x for x in label_indx if len(x) > 0]

    spans = []
    for indx in clean_label_indx:
        if len(indx) == 1:
            span = words[indx[0]]
            label = labels[indx[0]].upper()
        else:
            span = ' '.join([words[i] for i in indx])  
            label = [labels[i].upper() for i in indx][0]
        #remove punctuation and strip whitespace for spans
        span_clean = span.strip()
        for m in re.finditer(re.escape(span_clean), sentence):
            spans.append({"start":m.start(), "end":m.end(), "entity": label, "text": m.group()})
    
    return spans

In [3]:
def transform_csv_annotated_to_json(input_path):
    DATA = []
    data = (pd.read_csv(input_path, encoding='ISO-8859-1')
          .fillna(method='ffill'))
    for sent, sent_info in data.groupby('Review #'):
      words = list(sent_info["Word"])
      #convert words to sentence and get rid of spaces between punctuation characters
      sentence = re.sub(r'\s([?.!"](?:\s|$))', r'\1', " ".join(words))
      #get labels
      labels = list(sent_info['Tag'])
      #identify token span start, span ends and span category
      span_ents = get_span_indx(labels, words, sentence)
      DATA.append({"TEXT": sentence, "ENTITIES": span_ents})
    return DATA


In [4]:
data_csv = transform_csv_annotated_to_json("testing-data/product_reviews9.csv")
with open('testing-data/true_data.json', 'w') as fp:
    json.dump(data_csv, fp)
print(data_csv[1999])

{'TEXT': 'Hi my name is Sofia thompson. I hate the Xiaomi , the design is awful. I expected more. My social sn is 068289873 and my credit number is 4881 8730 3709 2414', 'ENTITIES': [{'start': 14, 'end': 28, 'entity': 'PERSON', 'text': 'Sofia thompson'}, {'start': 104, 'end': 113, 'entity': 'US_SSN', 'text': '068289873'}, {'start': 138, 'end': 157, 'entity': 'CREDIT_CARD', 'text': '4881 8730 3709 2414'}]}


### Extract only sentences to send to the model

In [70]:
def extract_sentences(input_file, output_file):
  data = (pd.read_csv(input_file, encoding='ISO-8859-1')
    .fillna(method='ffill'))
  with open(output_file, 'w', encoding='ISO-8859-1') as fo:
    writer = csv.writer(fo)
    writer.writerow(['SENTENCES']) 
    for sent, sent_info in data.groupby('Review #'):
      words = list(sent_info["Word"])
      sentence = re.sub(r'\s([?.!"](?:\s|$))', r'\1', " ".join(words))
      writer.writerow([sentence])

In [50]:
extract_sentences("testing-data/product_reviews9.csv", "testing-data/sentences_evaluate.csv")

### RUN THE MODEL WITH THE EXTRACTED SENTENCES

In [19]:
model_results(csv_path="testing-data/sentences_evaluate.csv", json_path="testing-data/output_custom_P_E.json", 
entities=entities, threshold=threshold, analyzer=analyzer, columns=["SENTENCES"], check_overlaps=True)

100%|██████████| 2000/2000 [02:01<00:00, 16.47it/s]


### EVALUATING

In [23]:
def generate_array_evaluate_per_words(array):
  """Generate array for evaluation."""
  array_evaluate = []
  text = array['TEXT']
  for word in text.split(' '):
    is_in = False
    for entity in array["ENTITIES"]:
      if word in entity["text"] and entity["entity"] != "LOCATION":
        is_in = True
        break
    if not is_in:
      array_evaluate.append("O")
  return array_evaluate


In [38]:
def calculate_precision_recall_for_entities(ground_truth, predictions, output_file):
    """Calculate precision and recall."""
    gt_eval_array = []
    pr_eval_array = []
    for gt in ground_truth:
        a_eval = generate_array_evaluate_per_words(gt)
        gt_eval_array.append(a_eval)
    for pr in predictions:
        a_eval = generate_array_evaluate_per_words(pr)
        pr_eval_array.append(a_eval)
    # for i in range(2):
    #     print(ground_truth[i])
    #     print(gt_eval_array[i])
    #     print(pr_eval_array[i])
    # print("General Precision: ", accuracy_score(gt_eval_array, pr_eval_array))
    # print("General Recall: ", recall_score(gt_eval_array, pr_eval_array))
    # print("General F1: ", f1_score(gt_eval_array, pr_eval_array))
    report = classification_report(gt_eval_array, pr_eval_array)
    
    with open(output_file, 'w', encoding='ISO-8859-1') as f:
        f.write("General Precision: " + str(accuracy_score(gt_eval_array, pr_eval_array)) + "\n")
        f.write("General Recall: " + str(recall_score(gt_eval_array, pr_eval_array)) + "\n")
        f.write("General F1: " + str(f1_score(gt_eval_array, pr_eval_array)) + "\n")
        f.write(report)

In [5]:
with open("testing-data/output_roberta.json", "r") as f:
  json_data = f.read()
prediction_data = json.loads(json_data)

In [6]:
# Remove unwanted entities from predictions_all
entities_to_remove = ['ORGANIZATION', 'AGE', 'DATE_TIME', 'O']
for i in range(len(prediction_data)):
    entities = prediction_data[i]['ENTITIES']
    for j in range(len(entities)-1, -1, -1):
        if entities[j]['entity'] in entities_to_remove:
            del entities[j]

In [25]:
warnings.filterwarnings('ignore', '.* seems not to be NE tag\.')
calculate_precision_recall_for_entities(data_csv, prediction_data, "eval/REPORT_custom_P_E.txt")

/Users/gonzalo.zelinka/Desktop/PII_POC/.venv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
def calculate_precision_recall_for_sentence(ground_truth_all, predictions_all, output_file):
    """Extract entities from ground truth and predictions."""
    gt_eval_array = []
    pr_eval_array = []

    for i in range(len(ground_truth_all)):
        test = ground_truth_all[i]
        test1 = predictions_all[i]
        entities_gt = sorted(set(item['entity'] for item in test['ENTITIES']))
        entities_pr = sorted(set(item['entity'] for item in test1['ENTITIES']))
        
        for entity in entities_gt:
            if entity not in entities_pr:
                entities_pr.insert(entities_gt.index(entity), 'O')
                print(f"Warning: entity {entity} not found in sample {i}")

        for entity in entities_pr:
            if entity not in entities_gt:
                entities_pr.insert(entities_pr.index(entity), 'O')
                print(f"Warning: entity {entity} not found in sample {i}")
        
        print(f"Ground truth: {entities_gt}")
        print(f"Predictions: {entities_pr}")

        if len(entities_gt) != len(entities_pr):
            print(f"Warning: inconsistent number of entities in sample {i}")
            print(f"Ground truth: {entities_gt}")
            print(f"Predictions: {entities_pr}")
            continue
        
        gt_eval_array.append(entities_gt)
        pr_eval_array.append(entities_pr)
    
    report = classification_report(gt_eval_array, pr_eval_array)
    
    with open(output_file, 'w', encoding='ISO-8859-1') as f:
        f.write("General Precision: " + str(accuracy_score(gt_eval_array, pr_eval_array)) + "\n")
        f.write("General Recall: " + str(recall_score(gt_eval_array, pr_eval_array)) + "\n")
        f.write("General F1: " + str(f1_score(gt_eval_array, pr_eval_array)) + "\n")
        f.write(report)

In [19]:
def get_unique_entities(arrays):
    unique_entities = set()
    for array in arrays:
        unique_entities |= set(array)
    return sorted(list(unique_entities))

In [8]:
calculate_precision_recall_for_sentence(data_csv[0:2], prediction_data[0:2], "eval/REPORT_SENTENCE.txt")

In [31]:
print(data_csv[7]["ENTITIES"])
print(prediction_data[7]["ENTITIES"])

[{'start': 21, 'end': 33, 'entity': 'PERSON', 'text': 'Zoey Edwards'}, {'start': 129, 'end': 151, 'entity': 'EMAIL_ADDRESS', 'text': 'edwards-zoey@gmail.com'}, {'start': 190, 'end': 201, 'entity': 'LOCATION', 'text': '900 F St NW'}, {'start': 213, 'end': 222, 'entity': 'US_SSN', 'text': '367245504'}, {'start': 252, 'end': 271, 'entity': 'CREDIT_CARD', 'text': '2259-8740-7030-1462'}]
[{'start': 21, 'end': 33, 'entity': 'PERSON', 'text': 'Zoey Edwards'}, {'start': 129, 'end': 151, 'entity': 'EMAIL_ADDRESS', 'text': 'edwards-zoey@gmail.com'}, {'start': 190, 'end': 195, 'entity': 'LOCATION', 'text': '900 F'}, {'start': 196, 'end': 201, 'entity': 'LOCATION', 'text': 'St NW'}, {'start': 213, 'end': 222, 'entity': 'US_SSN', 'text': '367245504'}, {'start': 252, 'end': 255, 'entity': 'PHONE_NUMBER', 'text': '225'}, {'start': 255, 'end': 271, 'entity': 'PHONE_NUMBER', 'text': '9-8740-7030-1462'}]
